# WIDER Datset

## Downloading and unzipping

In [ ]:
!gdown https://drive.google.com/u/0/uc?id=0B6eKvaijfFUDQUUwd21EckhUbWs&export=download
!gdown https://drive.google.com/u/0/uc?id=0B6eKvaijfFUDd3dIRmpvSk8tLUk&export=download

In [ ]:
!wget http://mmlab.ie.cuhk.edu.hk/projects/WIDERFace/support/bbx_annotation/wider_face_split.zip

In [ ]:
!cp /content/drive/MyDrive/WIDER_Dataset/* /content/

In [ ]:
!unzip -qx /content/WIDER_train.zip
!unzip -qx /content/WIDER_val.zip
!unzip -qx /content/wider_face_split.zip

## Creating the dataset

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from glob import glob as g
import cv2
from tqdm.notebook import tqdm
from shutil import copy, move
from google.colab.files import download

In [ ]:
# Val Dataset
new_imgs_dir = '/content/newDataset/images/val'
new_lbls_dir = '/content/newDataset/labels/val'
label_text_name = '/content/wider_face_split/wider_face_val_bbx_gt.txt'
imgs_address = '/content/WIDER_val/images'

os.makedirs(new_imgs_dir,exist_ok = True)
os.makedirs(new_lbls_dir,exist_ok = True)
annots = open(label_text_name) 
lines = annots.readlines()
names =   [x for x in lines if 'jpg' in x]
indices = [lines.index(x) for x in names]


for n in tqdm(range(len(names[:]))):
    i = indices[n]
    name = lines[i].rstrip()
    old_img_path = os.path.join(imgs_address , name)
    name = name.split('/')[-1]
    label_path = os.path.join(new_lbls_dir , name.split('.')[0] + '.txt')
    img_path = os.path.join(new_imgs_dir , name)
    
    num_objs = int(lines[i+1].rstrip())
    bboxs = lines[i+2 : i+2+num_objs]
    bboxs = list(map(lambda x:x.rstrip() , bboxs))
    bboxs = list(map(lambda x:x.split()[:4], bboxs))
    # if len(bboxs) > 5:
    #     continue
    img = cv2.imread(old_img_path)
    img_h,img_w,_ = img.shape
    img_h,img_w,_ = img.shape
    f = open(label_path, 'w')
    count = 0 # Num of bounding box
    for bbx in bboxs:
        x1 = int(bbx[0])
        y1 = int(bbx[1])
        w = int(bbx[2])
        h = int(bbx[3])
    #     #yolo:
        x = (x1 + w//2) / img_w
        y = (y1 + h//2) / img_h
        w = w / img_w
        h = h / img_h
        if w * h * 100 > 2:
            yolo_line = f'{0} {x} {y} {w} {h}\n'
            f.write(yolo_line)
            count += 1
    f.close()
    if count > 0:   
        copy(old_img_path , img_path)
    else:
        os.remove(label_path)

In [ ]:
# Train Dataset
new_imgs_dir = '/content/newDataset/images/train'
new_lbls_dir = '/content/newDataset/labels/train'
label_text_name = '/content/wider_face_split/wider_face_train_bbx_gt.txt'
imgs_address = '/content/WIDER_train/images'

os.makedirs(new_imgs_dir,exist_ok = True)
os.makedirs(new_lbls_dir,exist_ok = True)
annots = open(label_text_name) 
lines = annots.readlines()
names =   [x for x in lines if 'jpg' in x]
indices = [lines.index(x) for x in names]


for n in tqdm(range(len(names[:]))):
    i = indices[n]
    name = lines[i].rstrip()
    old_img_path = os.path.join(imgs_address , name)
    name = name.split('/')[-1]
    label_path = os.path.join(new_lbls_dir , name.split('.')[0] + '.txt')
    img_path = os.path.join(new_imgs_dir , name)
    num_objs = int(lines[i+1].rstrip())
    bboxs = lines[i+2 : i+2+num_objs]
    bboxs = list(map(lambda x:x.rstrip() , bboxs))
    bboxs = list(map(lambda x:x.split()[:4], bboxs))
    # if len(bboxs) > 5:
    #     continue
    img = cv2.imread(old_img_path)
    img_h, img_w, _ = img.shape
    f = open(label_path, 'w')
    count = 0 # Num of bounding box
    for bbx in bboxs:
        x1 = int(bbx[0])
        y1 = int(bbx[1])
        w = int(bbx[2])
        h = int(bbx[3])
    #     #yolo:
        x = (x1 + w//2) / img_w
        y = (y1 + h//2) / img_h
        w = w / img_w
        h = h / img_h
        if w * h * 100 > 2:
            yolo_line = f'{0} {x} {y} {w} {h}\n'
            f.write(yolo_line)
            count += 1
    f.close()
    if count > 0:   
        copy(old_img_path , img_path)
    else:
        os.remove(label_path)

## Resizing and Showing

In [ ]:
def resize_img(input_name , output_name, target_width = 640):
    im = cv2.imread(input_name)
    h,w,_  = im.shape
    target_height = int(h / w * target_width)
    im = cv2.resize(im , (target_width , target_height), interpolation = cv2.INTER_AREA)
    cv2.imwrite(output_name , im)

def resize_all_imgs(imgs_dir):
    names = g(os.path.join(imgs_dir , '*'))
    for img in tqdm(names):
        resize_img(img, img)

In [ ]:
names = g('/content/newDataset/labels/*/*')
print(f'Threre are {len(names)}  images')

In [ ]:
resize_all_imgs('/content/newDataset/images/*')

In [ ]:
n = np.random.randint(0, len(names))
f = open(names[n])

lines = f.readlines()

In [ ]:
lines

In [ ]:
n = np.random.randint(0, len(names))
f = open(names[n])

lines = f.readlines()
classes = list(map(lambda x: int(x[0]), lines))
lines = list(map(lambda x:x.rstrip()[2:], lines))
objects = list(map(lambda x:(x.split()), lines))

img = cv2.imread(names[n].replace('txt','jpg').replace('labels', 'images'))
for c, bbox in zip(classes, objects):
  bbox = list(map(lambda x:float(x), bbox))
  x,y,w,h = bbox
  img_h = img.shape[0]
  img_w = img.shape[1]
  x = int(x * img_w)
  w = int(w * img_w)
  y = int(y * img_h)
  h = int(h * img_h)
  color = (255,100,50)
  cv2.rectangle(img , (int(x-w/2), int(y-h/2)), (int(x+w/2), int(y+h/2)), color , 4)
plt.figure(figsize = (8,8))
plt.imshow(img[:,:,::-1]); plt.axis('off')
print(f'number of bounding boxes : {len(classes)}')
print(f'Shape on the image : {img.shape}')

## Create Grid

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np

images = []
for _ in range(25):
    n = np.random.randint(0, len(names))
    f = open(names[n])

    lines = f.readlines()
    classes = list(map(lambda x: int(x[0]), lines))
    lines = list(map(lambda x:x.rstrip()[2:], lines))
    objects = list(map(lambda x:(x.split()), lines))

    img = cv2.imread(names[n].replace('txt','jpg').replace('labels', 'images'))
    for c, bbox in zip(classes, objects):
        bbox = list(map(lambda x:float(x), bbox))
        x,y,w,h = bbox
        img_h = img.shape[0]
        img_w = img.shape[1]
        x = int(x * img_w)
        w = int(w * img_w)
        y = int(y * img_h)
        h = int(h * img_h)
        color = (255,100,50)
        cv2.rectangle(img , (int(x-w/2), int(y-h/2)), (int(x+w/2), int(y+h/2)), color , 6)
    # plt.figure(figsize = (8,8))
    # plt.imshow(img[:,:,::-1]); plt.axis('off')
    # print(f'number of bounding boxes : {len(classes)}')
    images.append(img[:,:,::-1])
fig = plt.figure(figsize=(16., 16.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(5 ,5),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )

for ax, im in zip(grid, images):
    # Iterating over the grid returns the Axes.
    ax.imshow(im)
    ax.axis('off')

plt.show()

In [ ]:
fig.savefig('grid_output.png')
download('grid_output.png')

## Train/Val >>>> ZIP

In [ ]:
os.makedirs('/content/Yolo/images', exist_ok= True)
os.makedirs('/content/Yolo/labels', exist_ok= True)
os.makedirs('/content/Yolo/images/train', exist_ok= True)
os.makedirs('/content/Yolo/images/val', exist_ok= True)
os.makedirs('/content/Yolo/labels/train', exist_ok= True)
os.makedirs('/content/Yolo/labels/val', exist_ok= True)

In [ ]:
labels_address = '/content/newDataset/labels/'
imgs_address = '/content/newDataset/images/'
np.random.seed(101)
names = os.listdir(imgs_address)
randvec = np.random.rand(len(names))
i = 0
for name in tqdm(names[:]):

  epsilon = randvec[i]
  i += 1
  epsilon = np.random.rand(1)
  if epsilon>0.85: #Validation
    copy(imgs_address + name , '/content/Yolo/images/val/' + name)
    copy(labels_address + name.split('.')[0] + '.txt' , '/content/Yolo/labels/val/' + name.split('.')[0] + '.txt')
    
  
  else: #Train
    copy(imgs_address + name,  '/content/Yolo/images/train/' + name)
    copy(labels_address + name.split('.')[0] + '.txt' , '/content/Yolo/labels/train/' + name.split('.')[0] + '.txt')



In [ ]:
!zip -rq Yolo.zip /content/Yolo

# Training Yolo

In [ ]:
import torch
from IPython.display import clear_output
torch.cuda.get_device_name()

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -qr /content/yolov5/requirements.txt
%cd yolov5
clear_output()  
f = open('/content/yolov5/data/dataset.yaml', 'w')
f.write('train: /content/newDataset/images/train')
f.write('\nval: /content/newDataset/images/val')
f.write('\nnc: {}'.format(1))
f.write("\nnames: ['Face']")

f.close()

# f = open('/content/yolov5/models/newyolov5s.yaml', 'w')
# f.write('nc: {}\n'.format(1))
# f.write('\n'.join(open('/content/yolov5/models/yolov5s.yaml').read().split('\n')[2:]))
# f.close()

In [ ]:
!python train.py --img 640 --batch 64 --workers 8 --epochs 300\
  --weights yolov5s.pt\
  --cfg /content/yolov5/models/yolov5s.yaml\
  --data /content/yolov5/data/dataset.yaml\
  --weights yolov5s\